In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from PIL import Image
import os
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from keras.models import Sequential , load_model
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [18]:
cd /content/gdrive/MyDrive

/content/gdrive/MyDrive


In [ ]:
import os
import shutil

# Define the path to the original dataset
data_path = '/content/gdrive/MyDrive/AICapstoneProjectData/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone/CT-KIDNEY-DATASET-Normal-Cyst-Tumor-Stone'
# Define the path to the output directory
output_path = './dataset'

# Define the ratios for train/val/test splits
train_ratio = 0.8
test_ratio = 0.2

# Create the output directories if they do not exist
if not os.path.exists(output_path):
    os.makedirs(output_path)
for dir_name in ['train', 'test']:
    dir_path = os.path.join(output_path, dir_name)
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
    for class_name in ['Cyst','Normal','Stone','Tumor']:
        class_path = os.path.join(dir_path, class_name)
        if not os.path.exists(class_path):
            os.makedirs(class_path)

# Loop over each class and copy files to train/val/test directories
for class_name in ['Cyst', 'Normal', 'Stone', 'Tumor']:
    class_path = os.path.join(data_path, class_name)
    files = os.listdir(class_path)
    num_files = len(files)
    num_train = int(num_files * train_ratio)
    num_test = num_files - num_train
    for i, file_name in enumerate(files):
        src_path = os.path.join(class_path, file_name)
        if i < num_train:
            dst_path = os.path.join(output_path, 'train', class_name, file_name)
        else:
            dst_path = os.path.join(output_path, 'test', class_name, file_name)
        shutil.copy(src_path, dst_path)

In [23]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.efficientnet import preprocess_input

# Set up the image data generators
train_path = './dataset/train'
test_path = './dataset/test'
img_size = (224,224)
batch_size = 10
test_batch_size = 10
test_steps = 1

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    shear_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    preprocessing_function=preprocess_input,
)
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input
)

train_generator = train_datagen.flow_from_directory(
    directory=train_path,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=True,
    batch_size=batch_size
)
test_generator = test_datagen.flow_from_directory(
    directory=test_path,
    target_size=(224, 224),
    class_mode='categorical',
    shuffle=False,
    batch_size=test_batch_size
)


Found 9955 images belonging to 4 classes.
Found 2492 images belonging to 4 classes.


In [24]:
# Load the preprocessed image data and labels into numpy arrays
train_data = train_generator.next()[0]
train_labels = train_generator.next()[1]
test_data = test_generator.next()[0]
test_labels = test_generator.next()[1]

# Reshape the numpy arrays to flatten the images
train_data = train_data.reshape(train_data.shape[0], -1)
test_data = test_data.reshape(test_data.shape[0], -1)

# Instantiate the logistic regression model
logreg = LogisticRegression(solver='liblinear')

# Fit the logistic regression model to the training data
logreg.fit(train_data, np.argmax(train_labels, axis=1))

# Predict the labels of the test data using the logistic regression model
train_predictions = logreg.predict(train_data)
test_predictions = logreg.predict(test_data)

# Calculate the accuracy of the logistic regression model on the test data
train_accuracy = np.mean(train_predictions == np.argmax(train_labels, axis=1))
test_accuracy = np.mean(test_predictions == np.argmax(test_labels, axis=1))

print("Logistic regression Training accuracy: {:.2f}%".format(train_accuracy*100))
print("Logistic regression Testing accuracy: {:.2f}%".format(test_accuracy*100))

Logistic regression Training accuracy: 100.00%
Logistic regression Testing accuracy: 10.00%


In [25]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
# Generate the confusion matrix
cm = confusion_matrix(np.argmax(test_labels,axis=1),test_predictions)
print("Confusion matrix:")
print(cm)

Confusion matrix:
[[1 9]
 [0 0]]


In [26]:
# Generate the classification report
cr = classification_report(np.argmax(test_labels,axis=1),test_predictions)
print("Classification report:")
print(cr)

Classification report:
              precision    recall  f1-score   support

           0       1.00      0.10      0.18        10
           1       0.00      0.00      0.00         0

    accuracy                           0.10        10
   macro avg       0.50      0.05      0.09        10
weighted avg       1.00      0.10      0.18        10



/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
